In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Import Wartung.csv data
data = pd.read_csv('https://raw.githubusercontent.com/Flitschi7/big-data-management/main/data/wartung.csv?token=GHSAT0AAAAAACMYJ6LRPC2SO7J2X7Z4WCGUZNXWCHA', sep=';', decimal=',', encoding='latin-1')

Hier muss noch eine Datenbank angebunden werden, um die Daten zu speichern.

## Datenaufbereiten

In [6]:
#Datentypen ausgeben
print("Datentypen:")
print(data.info())

Datentypen:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ï»¿MesswertID     1095 non-null   int64  
 1   MaschinenID       1095 non-null   object 
 2   Datum             1095 non-null   object 
 3   Zeit              1095 non-null   object 
 4   Druck             911 non-null    float64
 5   Temperatur        917 non-null    float64
 6   Vibration         915 non-null    float64
 7   Anzahlwarning     1095 non-null   int64  
 8   Ausschuss         803 non-null    float64
 9   Produktionsindex  1095 non-null   int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 85.7+ KB
None


In [7]:
#Anzahl missing data
print("Datentypen:")
print(data.isnull().sum())

Datentypen:
ï»¿MesswertID         0
MaschinenID           0
Datum                 0
Zeit                  0
Druck               184
Temperatur          178
Vibration           180
Anzahlwarning         0
Ausschuss           292
Produktionsindex      0
dtype: int64


In [8]:
#Datensätze mit Duplikaten entfernen
data.drop_duplicates(inplace=True)
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ï»¿MesswertID     1095 non-null   int64  
 1   MaschinenID       1095 non-null   object 
 2   Datum             1095 non-null   object 
 3   Zeit              1095 non-null   object 
 4   Druck             911 non-null    float64
 5   Temperatur        917 non-null    float64
 6   Vibration         915 non-null    float64
 7   Anzahlwarning     1095 non-null   int64  
 8   Ausschuss         803 non-null    float64
 9   Produktionsindex  1095 non-null   int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 85.7+ KB
None


### Fehlerhafte Daten  beim Datum identifizieren und korrigieren

In [9]:
# Convert date column to datetime format if value is not usable as date format remove row
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y', errors='coerce')

In [10]:
# Convert time column to datetime format if value is not usable as time format remove row
data['Zeit'] = pd.to_datetime(data['Zeit'], format='%H:%M:%S', errors='coerce')

In [11]:
d1 = data

### Neue Spalten hinzufügen

In [12]:
# Add new Column 'Ausfall' with 0 or 1 for each row. Ausfall = 1 means Column Druck, Temperatur Vibration and Ausschuss are all NaN
d1['Ausfall'] = np.where( d1['Druck'].isnull() & d1['Temperatur'].isnull() & d1['Vibration'].isnull() & d1['Ausschuss'].isnull(), 1, 0)
print(d1.head(10))
d2 = d1

   ï»¿MesswertID MaschinenID      Datum Zeit      Druck  Temperatur  \
0          12478          A1 2022-02-01  NaT  47.619048   61.904762   
1          12479          A1 2022-02-01  NaT  48.571429   62.857143   
2          12480          A1 2022-02-01  NaT  57.142857   76.190476   
3          12481          A1 2022-02-01  NaT        NaN         NaN   
4          12482          A1 2022-02-01  NaT        NaN         NaN   
5          12483          A1 2022-02-01  NaT        NaN         NaN   
6          12484          A1 2022-02-01  NaT        NaN         NaN   
7          12485          A1 2022-02-01  NaT        NaN         NaN   
8          12486          A1 2022-02-01  NaT        NaN         NaN   
9          12487          A1 2022-02-01  NaT        NaN         NaN   

   Vibration  Anzahlwarning  Ausschuss  Produktionsindex  Ausfall  
0  28.571429              3        2.0                85        0  
1  47.619048              5        3.0                87        0  
2  33.333333  

### Fehlende Daten ergänzen

In [13]:
# If Ausfall = 1 and 'Druck' is missing then "Error"
d2['Druck'] = np.where( d2['Ausfall'] == 1 & d2['Druck'].isnull(), "Error", d2['Druck'])
d3 = d2


In [14]:
# If Ausfall = 1 and 'Temperatur' is missing then "Error"
d3['Temperatur'] = np.where( d3['Ausfall'] == 1 & d3['Temperatur'].isnull(), "Error", d3['Temperatur'])
d4 = d3

In [15]:
# If Ausfall = 1 and 'Vibration' is missing then "Error"
d4['Vibration'] = np.where( d4['Ausfall'] == 1 & d4['Vibration'].isnull(), "Error", d4['Vibration'])
d5 = d4

In [16]:
# If Ausfall = 1 and 'Ausschuss' is missing then "Error"
d5['Ausschuss'] = np.where( d5['Ausfall'] == 1 & d5['Ausschuss'].isnull(), "Error", d5['Ausschuss'])

Gegebenenfalls können anstelle von Entfernen auch Werte durch synthetische Daten ersetzt werden

In [17]:
# Remove all rows with Druck, Temperatur, Vibration or Ausschuss = Error
d5 = d5[d5.Druck != 'Error']
d5 = d5[d5.Temperatur != 'Error']
d5 = d5[d5.Vibration != 'Error']
d5 = d5[d5.Ausschuss != 'Error']